In [1]:
from pathlib import Path
import os

yr = 2023
d = 7

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  hands = []
  bids = []
  for l in inp.split('\n'):
    hands.append(l.split(' ')[0])
    bids.append(int(l.split(' ')[1]))
  return {'hands': hands, 'bids': bids}

In [3]:
def hand_to_counts(h):
  seen_cards = []
  counts = []
  for c in h:
    if c not in seen_cards:
      seen_cards.append(c)
      counts.append(1)
    else:
      ci = seen_cards.index(c)
      counts[ci] += 1
  return list(sorted(counts, reverse=True))

def is_5_of_kind(h):
  cnts = hand_to_counts(h)
  return cnts[0]==5

def is_4_of_kind(h):
  cnts = hand_to_counts(h)
  return cnts[0]==4

def is_full_house(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==3 and hand_to_counts(h)[1]==2

def is_3_of_kind(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==3 and hand_to_counts(h)[1]!=2

def is_2_pair(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==2 and hand_to_counts(h)[1]==2

def is_1_pair(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==2 and hand_to_counts(h)[1]!=2

def is_high_card(h):
  cnts = hand_to_counts(h)
  return hand_to_counts(h)[0]==1

def generate_hand_to_priority(jokers_wild=False):
  from itertools import product
  from copy import deepcopy
  from functools import cmp_to_key
  from tqdm import tqdm

  def compare_hands_by_lead_cards(h1, h2):
    if not jokers_wild:
      card_order = {
                    'A':12,
                    'K':11,
                    'Q':10,
                    'J':9,
                    'T':8,
                    '9':7,
                    '8':6,
                    '7':5,
                    '6':4,
                    '5':3,
                    '4':2,
                    '3':1,
                    '2':0
                  }
    else:
      card_order = {
              'A':12,
              'K':11,
              'Q':10,
              'T':9,
              '9':8,
              '8':7,
              '7':6,
              '6':5,
              '5':4,
              '4':3,
              '3':2,
              '2':1,
              'J':0,
            }
    for i in range(5):
      if card_order[h1[i]] < card_order[h2[i]]:
        return -1
      elif card_order[h1[i]] > card_order[h2[i]]:
        return 1
    return 0

  def get_highest_priority_hand_type(hand):


    prio = {'4_of_kind': 5,
            'full_house': 4,
            '3_of_kind': 3,
            '2_pair': 2,
            '1_pair': 1,
            'high_card': 0}

    hand = list(hand)
    hands_to_check = [hand]

    # If the jokers are wild then we will check all possible hands
    # that can be generated from replacing the jokers in the hand
    if jokers_wild:
      joker_locs = [i for i, c in enumerate(hand) if c=='J']
      potential_new_jokers = product('AKQJT98765432', repeat=len(joker_locs))
      for pnj in potential_new_jokers:
        new_hand = deepcopy(hand)
        for i, jl in enumerate(joker_locs):
          new_hand[jl] = pnj[i]
        hands_to_check.append(new_hand)


    hand_types = []
    for h in hands_to_check:
      if is_5_of_kind(h):
        return '5_of_kind'
      elif is_4_of_kind(h):
        hand_types.append('4_of_kind')
      elif is_full_house(h):
        hand_types.append('full_house')
      elif is_3_of_kind(h):
        hand_types.append('3_of_kind')
      elif is_2_pair(h):
        hand_types.append('2_pair')
      elif is_1_pair(h):
        hand_types.append('1_pair')
      elif is_high_card(h):
        hand_types.append('high_card')
      else:
        raise Exception(h, " COULDN'T BE CLASSIFIED")

    return max(hand_types, key=lambda x: prio[x])


  # We will build lists of all possible hands,
  # sort them by lead cards, and then use them to determine
  # overall priority for every possible hand
  five_of_kinds = []
  four_of_kinds = []
  full_houses = []
  three_of_kinds = []
  two_pairs = []
  one_pairs = []
  high_cards = []


  possible_combos = product('AKQJT98765432', repeat=5)
  for pp in tqdm(list(possible_combos)):
    hand_type = get_highest_priority_hand_type(pp)
    if hand_type == '5_of_kind':
      five_of_kinds.append(pp)
    elif hand_type == '4_of_kind':
      four_of_kinds.append(pp)
    elif hand_type == 'full_house':
      full_houses.append(pp)
    elif hand_type == '3_of_kind':
      three_of_kinds.append(pp)
    elif hand_type == '2_pair':
      two_pairs.append(pp)
    elif hand_type == '1_pair':
      one_pairs.append(pp)
    elif hand_type == 'high_card':
      high_cards.append(pp)



  # Once we know which hands belong to which category, we just need
  # to sort each category by the lead cards
  five_of_kinds = sorted(five_of_kinds,
                         key=cmp_to_key(compare_hands_by_lead_cards))
  four_of_kinds = sorted(four_of_kinds,
                         key=cmp_to_key(compare_hands_by_lead_cards))
  full_houses = sorted(full_houses,
                       key=cmp_to_key(compare_hands_by_lead_cards))
  three_of_kinds = sorted(three_of_kinds,
                          key=cmp_to_key(compare_hands_by_lead_cards))
  two_pairs = sorted(two_pairs,
                     key=cmp_to_key(compare_hands_by_lead_cards))
  one_pairs = sorted(one_pairs,
                     key=cmp_to_key(compare_hands_by_lead_cards))
  high_cards = sorted(high_cards,
                      key=cmp_to_key(compare_hands_by_lead_cards))

  h2p = {}
  for i, h in enumerate(high_cards + one_pairs + two_pairs + three_of_kinds +
                        full_houses + four_of_kinds + five_of_kinds):
    h2p[''.join(h)] = i

  return h2p

In [4]:
def total_winnings(formatted_input, hand_to_priority):
  hb = list(zip(formatted_input['hands'], formatted_input['bids']))
  hb.sort(key=lambda x: hand_to_priority[x[0]])
  return sum([x[1]*(i+1) for i, x in enumerate(hb)])

### A faster way to do this would be to just generate the hand_to_priorities maps only for the hands that we have in the input.

##### I felt like doing it this way though so that any hypothetical future hands could be easily looked up

In [5]:
import time

t = time.time()

formatted_input = format_input(inp)

h2p = generate_hand_to_priority()
h2pjw = generate_hand_to_priority(jokers_wild=True)

tt = time.time()

print(total_winnings(formatted_input, h2p))
print(total_winnings(formatted_input, h2pjw))


print("\nRUNTIME (GENERATE ALL POSSIBLE HAND PRIORITIES): ", tt-t)
print("RUNTIME (CALCULATE WINNINGS): ", time.time()-tt)
print("RUNTIME (TOTAL): ", time.time()-t)

100%|████████████████████████████████████████████████████████████████████████| 371293/371293 [02:42<00:00, 2283.36it/s]


250602641
251037509

RUNTIME (GENERATE ALL POSSIBLE HAND PRIORITIES):  170.74941039085388
RUNTIME (CALCULATE WINNINGS):  0.0009961128234863281
RUNTIME (TOTAL):  170.75040650367737
